# HTTP client

In [ ]:
!pip install dicttoxml

In [ ]:
import requests
import os

class HttpClient:
    def __init__(self, file_path='./src/main/resources/application.properties'):
        self.session = requests.Session()
        self.session.headers.update({
            "Content-Type": "application/json",
            "X-Requested-With": "XMLHttpRequest"
        })

        creds = self.read_credentials(file_path)
        self.base_url = creds.get('cyoda.host')
        self.username = creds.get('cyoda.name')
        self.password = os.getenv('DEMO_USER_PASSWD', 'default_password')

        self.authenticate()

    @staticmethod
    def read_credentials(file_path):
        credentials = {}
        try:
            with open(file_path, 'r') as file:
                for line in file:
                    key, value = line.strip().split('=')
                    credentials[key] = value
        except FileNotFoundError:
            raise Exception(f"File {file_path} not found")
        except ValueError:
            raise Exception(f"Error parsing credentials in {file_path}")
        return credentials

    def authenticate(self):
        if not self.base_url or not self.username or not self.password:
            raise Exception("Base URL, username, or password is missing")

        url = f"{self.base_url}/auth/login"
        response = self.session.post(url, json={"username": self.username, "password": self.password})
        response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code
        token = response.json().get('token')
        if token:
            self.session.headers.update({"Authorization": f"Bearer {token}"})
        else:
            raise Exception("Authentication failed, token not found")

    def request(self, method, endpoint, **kwargs):
        url = f"{self.base_url}{endpoint}"

        if 'json' in kwargs:
            self.session.headers.update({"Content-Type": "application/json"})
        elif 'data' in kwargs:
            data = kwargs['data']
            if isinstance(data, str) and data.strip().startswith('<'):
                self.session.headers.update({"Content-Type": "application/xml"})
            else:
                self.session.headers.update({"Content-Type": "application/x-www-form-urlencoded"})
        else:
            self.session.headers.pop("Content-Type", None)

        try:
            response = self.session.request(method, url, **kwargs)
            response.raise_for_status()

            if response.content.strip():
                try:
                    return response.json()
                except ValueError:
                    print("Response is not JSON, returning as text.")
                    return response.text
            else:
                print("Request was successful, but the response is empty.")
                return None
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}")
            print(f"Response content: {response.text}")
        except ValueError as json_err:
            print(f"JSON parsing error occurred: {json_err}")
            print(f"Response content: {response.text}")
            return None
        except Exception as err:
            print(f"Other error occurred: {err}")

In [ ]:
client = HttpClient()
print("Client initialized and authenticated.")


In [ ]:
# PROVIDE DATA
import json
import dicttoxml
import os

entityName = "expense_report"
modelVersion = "1"
entityType = "TREE" # or "TABLE"
format = "JSON" # or "XML"
entityClassName = "com.cyoda.tdb.model.treenode.TreeNodeEntity"

json_file_path = os.path.join(os.getcwd(), 'sample_data.json')
xml_file_path = os.path.join(os.getcwd(), 'sample_data.xml')

new_data = None

if format == "JSON":
    try:
        with open(json_file_path, 'r') as json_file:
            new_data = json.load(json_file)
            print("Loaded JSON data:", new_data)
    except FileNotFoundError:
        print(f"File {json_file_path} not found.")
    except json.JSONDecodeError:
        print("Error decoding JSON from the file.")

elif format == "XML":
    try:
        with open(xml_file_path, 'r') as xml_file:
            new_data = xml_file.read()
            print("Loaded XML data as string:", new_data)
    except FileNotFoundError:
        print(f"File {xml_file_path} not found.")
    except Exception as e:
        print("Error reading XML from the file:", e)

else:
    print("Unsupported format. Please use 'JSON' or 'XML'.")

# ModelController

In [ ]:
# SAVE MODEL
converter = "SAMPLE_DATA"
url = f"/treeNode/model/import/{format}/{converter}/{entityName}/{modelVersion}"

if format.upper() == "JSON":
    response = client.request("POST", url, json=new_data)
elif format.upper() == "XML":
    xml_data = dicttoxml.dicttoxml(new_data, custom_root='root', attr_type=False).decode()
    response = client.request("POST", url, data=xml_data)
else:
    print("Unsupported format. Please use 'JSON' or 'XML'.")
    response = None

if response:
    print("Response:", response)

In [ ]:
# GET MODEL
converter = "JSON_SCHEMA"
#converter = "SIMPLE_VIEW"
url = f"/treeNode/model/export/{converter}/{entityName}/{modelVersion}"

response = client.request("GET", url)
print("Response:", response)

In [ ]:
# LOCK MODEL
url = f"/treeNode/model/{entityName}/{modelVersion}/lock"

response = client.request("PUT", url)
print("Response:", response)

In [ ]:
# UNLOCK MODEL
url = f"/treeNode/model/{entityName}/{modelVersion}/unlock"

response = client.request("PUT", url)
print("Response:", response)

In [ ]:
# GET ALL MODELS
url = f"/treeNode/model/"

response = client.request("GET", url)
print("Response:", response)

In [ ]:
# DELETE MODEL
url = f"/treeNode/model/{entityName}/{modelVersion}"

response = client.request("DELETE", url)
print("Response:", response)

# EntityController

In [ ]:
# SAVE A SINGLE ENTITY OR A COLLECTION OF ENTITIES OF THE SAME TYPE
url = f"/entity/{format}/{entityType}/{entityName}/{modelVersion}"

if format.upper() == "JSON":
    response = client.request("POST", url, json=new_data)
elif format.upper() == "XML":
    xml_data = dicttoxml.dicttoxml(new_data, custom_root='root', attr_type=False).decode()
    response = client.request("POST", url, data=xml_data)
else:
    print("Unsupported format. Please use 'JSON' or 'XML'.")
    response = None

if response:
    print("Response:", response)

In [ ]:
# SAVE A COLLECTION OF ENTITIES. Each entity can be from a different model and version
#url = f"/entity/{format}/{entityType}"

#if format.upper() == "JSON":
#    response = client.request("POST", url, json=new_data)
#elif format.upper() == "XML":
#    xml_data = dicttoxml.dicttoxml(new_data, custom_root='root', attr_type=False).decode()
#    response = client.request("POST", url, data=xml_data)
#else:
#    print("Unsupported format. Please use 'JSON' or 'XML'.")
#    response = None

#if response:
#    print("Response:", response)

In [ ]:
# GET ALL ENTITIES
url = f"/entity/{entityType}/{entityName}/{modelVersion}"

response = client.request("GET", url)
print("Response:", response)

In [ ]:
# GET ID OF THE FIRST ELEMENT (FROM RESPONSE OF # GET ALL ENTITIES)
if response and isinstance(response, list) and 'id' in response[0]:
    entityId = response[0]['id']
    print("ID of the first element:", entityId)
else:
    print("Response does not contain the expected data.")

In [ ]:
# GET ENTITY
url = f"/entity/{entityType}/{entityId}"

response = client.request("GET", url)
print("Response:", response)

In [ ]:
# UPDATE ENTITY
transition = "UPDATE"
url = f"/entity/{format}/{entityType}/{entityId}/{transition}"

response = client.request("PUT", url, json=new_data)
print("Response:", response)

In [ ]:
# GET CURRENT STATE OF AN ENTITY
url = f"/platform-api/entity-info/fetch/lazy?entityClass={entityClassName}&entityId={entityId}&columnPath=state"

response = client.request("GET", url)
if response and isinstance(response, list) and 'value' in response[0]:
    value = response[0]['value']
    print("State: ", value)
else:
    print("Response does not contain the expected 'value'.")

In [ ]:
# GET AVAILABLE TRANSITIONS
url = f"/platform-api/entity/fetch/transitions?entityId={entityId}&entityClass={entityClassName}"

response = client.request("GET", url)
print("Response:", response)

In [ ]:
# LAUNCH TRANSITION
transitionName = "SUBMIT"
url = f"/platform-api/entity/transition?entityId={entityId}&entityClass={entityClassName}&transitionName={transitionName}"

response = client.request("PUT", url)
print("Response:", response)

In [ ]:
# DELETE A SINGLE ENTITY BY ID
url = f"/entity/{entityType}/{entityId}"

response = client.request("DELETE", url)
print("Response:", response)

In [ ]:
# DELETE ALL ENTITIES FOR A GIVEN MODEL NAME AND VERSION
url = f"/entity/{entityType}/{entityName}/{modelVersion}"

response = client.request("DELETE", url)
print("Response:", response)